### GCN => GAT 

- GCN：Graph Conv Networks, GAT：Graph Attention Networks
- 都是用来学习 target node 的 representation
- gcn
    - $d_v$：表示节点的degree（即有多少条边连接），注意包含自己；
$$
\begin{align}
\mathbf{h}_{\mathcal{N}(v)} &= \sum_{u \in \mathcal{N}(v)} w_{u,v} \mathbf{h}_u \\
&= \sum_{u \in \mathcal{N}(v)} \sqrt{\frac{1}{d_v}} \sqrt{\frac{1}{d_u}} \mathbf{h}_u \\
&= \sqrt{\frac{1}{d_v}} \sum_{u \in \mathcal{N}(v)} \sqrt{\frac{1}{d_u}} \mathbf{h}_u
\end{align}
$$

- gat
    - https://www.youtube.com/watch?v=SnRfBfXwLuY

$$
\begin{align}
\mathbf{h}_{\mathcal{N}(v)} &= \sum_{u \in \mathcal{N}(v)} \underbrace{\text{softmax}_u \left( a(\mathbf{h}_u, \mathbf{h}_v) \right)}_{\alpha_{u,v}} \mathbf{h}_u \\
\alpha_{u,v} &= \frac{\exp\left(a(\mathbf{h}_u, \mathbf{h}_v)\right)}{\sum_{k \in \mathcal{N}(v)} \exp\left(a(\mathbf{h}_k, \mathbf{h}_v)\right)}
\end{align}
$$